In [1]:
from kernel import hadamard
import tilelang
import tilelang.language as T
# from fast_hadamard_transform import hadamard_transform
from ref import hadamard_transform_ref
import torch
b, d, dtype = 1024, 8192, torch.float32

In [2]:
def test(b=1024, d=8192, dtype=torch.float32):
    x = torch.rand(b, d, dtype=dtype, device='cuda')
    fn = hadamard(b, d, dtype.__str__().split('.')[-1])
    kernel = tilelang.compile(fn, out_idx=1, target='cuda')
    return torch.allclose(kernel(x), hadamard_transform_ref(x), rtol=1e-2, atol=1e-2)

In [3]:
test(d=8192)    

True

In [4]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    hadamard_transform(A)
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

/tmp/ipykernel_331891/876011834.py:1: FutureWarning: The attribute `use_cuda` will be deprecated soon, please use ``use_device = 'cuda'`` instead.
  with torch.autograd.profiler.profile(use_cuda=True) as prof:


NameError: name 'hadamard_transform' is not defined

In [ ]:
with torch.autograd.profiler.profile(use_cuda=True) as prof:
    kernel(A)
print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                     Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
              aten::empty         0.73%      19.481us         0.73%      19.481us      19.481us       3.000us       100.00%       3.000us       3.000us             1  
          cudaEventRecord         0.35%       9.423us         0.35%       9.423us       4.711us       0.000us         0.00%       0.000us       0.000us             2  
         cudaLaunchKernel         0.57%      15.173us         0.57%      15.173us      15.173us       0.000us         0.00%       0.000us       0.000us        

/tmp/ipykernel_175088/790097636.py:1: FutureWarning: The attribute `use_cuda` will be deprecated soon, please use ``use_device = 'cuda'`` instead.
  with torch.autograd.profiler.profile(use_cuda=True) as prof:
